In [ ]:
import os
import subprocess
import sys
from typing import List
from pyspark.sql import SparkSession
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the base directory
jars_home = '/workspace/delta-jars'

# Create SparkSession using the builder pattern
builder = (SparkSession.builder
           .appName("DeltaExample")
           .master("local[*]")
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
           .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
           .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
           .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
           .config("spark.hadoop.fs.s3a.path.style.access", "true")
           .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
           .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
           .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

           )

# Stop any existing session
if 'spark' in locals():
    spark.stop()


# Create the session
spark = builder.enableHiveSupport().getOrCreate()

# Initialize Delta Lake settings
# spark.sql("SET spark.databricks.delta.formatCheck.enabled=false")

# Access the SparkContext
# sc = spark.sparkContext

# Set the log level to INFO
# sc.setLogLevel("DEBUG")
# Get Hadoop configuration
hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
  
# Iterate and print all configuration entries
for entry in hadoop_conf.iterator():
    print(f"{entry.getKey()} = {entry.getValue()}")

# Test DataFrame
data = [(1, "John"), (2, "Jane")]
df = spark.createDataFrame(data, ["id", "name"])
df.show()


# First verify the S3 connection by listing the bucket
try:
    # Try to write to a simple parquet file first to test S3 connection
    # print("Testing S3 connection with parquet write...")
    # df.write.format("parquet").mode(
    #     "overwrite").save("s3a://wba/test4.parquet")
    # print("S3 connection successful")

    print("Attempting to write Delta table...")
    df.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .option("delta.compatibility.symlinkFormatManifest.enabled", "false") \
        .save("s3a://wba/example-table3")
    print("Successfully wrote Delta table")

except Exception as e:
    print(f"Error: {str(e)}")
    print("\nTrying local filesystem instead...")
    try:
        local_path = "/tmp/test-delta-table"
        df.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .save(local_path)
        print(f"Successfully wrote to {local_path}")
    except Exception as local_e:
        print(f"Error writing to local filesystem: {str(local_e)}")

your 131072x1 screen size is bogus. expect trouble
25/03/30 10:37:05 WARN Utils: Your hostname, JBLAPTOPW11 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/30 10:37:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/30 10:37:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


yarn.log-aggregation.file-formats = TFile
fs.s3a.select.output.csv.record.delimiter = \n
mapreduce.jobhistory.client.thread-count = 10
hadoop.security.group.mapping.ldap.posix.attr.uid.name = uidNumber
yarn.admin.acl = *
yarn.app.mapreduce.am.job.committer.cancel-timeout = 60000
yarn.federation.enabled = false
mapreduce.job.emit-timeline-data = false
fs.s3a.select.input.csv.quote.character = "
yarn.nodemanager.runtime.linux.sandbox-mode.local-dirs.permissions = read
yarn.resourcemanager.leveldb-state-store.path = ${hadoop.tmp.dir}/yarn/system/rmstore
ipc.client.connection.maxidletime = 10000
yarn.nodemanager.health-checker.scripts = script
yarn.nodemanager.process-kill-wait.ms = 5000
yarn.minicluster.use-rpc = false
io.map.index.interval = 128
mapreduce.task.profile.reduces = 0-2
hadoop.util.hash.type = murmur
yarn.webapp.api-service.enable = false
yarn.resourcemanager.nodemanagers.heartbeat-interval-min-ms = 1000
yarn.nodemanager.aux-services.manifest.reload-ms = 0
fs.s3a.path.style.a

+---+----+
| id|name|
+---+----+
|  1|John|
|  2|Jane|
+---+----+

Attempting to write Delta table...


25/03/30 10:37:25 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/03/30 10:37:25 WARN VersionInfoUtils: The AWS SDK for Java 1.x entered maintenance mode starting July 31, 2024 and will reach end of support on December 31, 2025. For more information, see https://aws.amazon.com/blogs/developer/the-aws-sdk-for-java-1-x-is-in-maintenance-mode-effective-july-31-2024/
You can print where on the file system the AWS SDK for Java 1.x core runtime is located by setting the AWS_JAVA_V1_PRINT_LOCATION environment variable or aws.java.v1.printLocation system property to 'true'.
This message can be disabled by setting the AWS_JAVA_V1_DISABLE_DEPRECATION_ANNOUNCEMENT environment variable or aws.java.v1.disableDeprecationAnnouncement system property to 'true'.
The AWS SDK for Java 1.x is being used here:
at java.base/java.lang.Thread.getStackTrace(Thread.java:1619)
at com.amazonaws.util.VersionInfoUtils.printDeprecationAn